# Skip-gram word2vec model 
Skip-gram model is trained to predict surrounding words given the current word.

*I love green eggs and ham.*

Assume a window size of three, this sentence can be broken down into the following sets of (context, word) pairs:

([I, green], love)

([love, eggs], green)

([green, and], eggs)

...

Since the skip-gram model predicts a context word given the center word, we can convert the preceding dataset to one of (input, output) pairs. That is, given an input word, we expect the skip-gram model to predict the output word:

(love, I), (love, green), (green, love), (green, eggs), (eggs, green), (eggs, and), …

We can also generate additional negative samples by pairing each input word with some random word in the vocabulary. For example:

(love, Sam), (love, zebra), (green, thing), …

Finally, we generate positive and negative examples for our classifier:

((love, I), 1), ((love, green), 1), …, ((love, Sam), 0), ((love, zebra), 0), …

![Skip-gram model](https://i.imgur.com/n78k91P.png)

In [ ]:
%matplotlib inline
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import nltk
import numpy as np
import operator

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 20

In [ ]:
lines = []
fin = open("alice_in_wonderland.txt", "r", encoding = 'utf-8')
for line in fin:
#############################################################################
# TODO: Write a for loop to read in the corpus.                             #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################
fin.close()

In [ ]:
#     line = line.strip()
#     if len(line) == 0:
#         continue
#     lines.append(line)

In [ ]:
sents = nltk.sent_tokenize(" ".join(lines))

#############################################################################
# TODO: Use tokenizer to tokenize the corpus.                               #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

xs = []
ys = []
for sent in sents:
#############################################################################
# TODO: Create training data for skipgram.                                  #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################


In [ ]:
# tokenizer = Tokenizer(5000)  # use top 5000 words only
# tokens = tokenizer.fit_on_texts(sents)
# vocab_size = len(tokenizer.word_counts) + 1

#     embedding = one_hot(sent, vocab_size)
#     triples = list(nltk.trigrams(embedding))
#     w_lefts = [x[0] for x in triples]
#     w_centers = [x[1] for x in triples]
#     w_rights = [x[2] for x in triples]
#     xs.extend(w_centers)
#     ys.extend(w_lefts)
#     xs.extend(w_centers)
#     ys.extend(w_rights)

In [ ]:
ohe = OneHotEncoder(n_values=vocab_size)
#############################################################################
# TODO: Use one hot encoder to fit on the corpus.                           #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
# X = ohe.fit_transform(np.array(xs).reshape(-1, 1)).todense()
# Y = ohe.fit_transform(np.array(ys).reshape(-1, 1)).todense()
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3,
#                                                 random_state=42)
# print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

In [ ]:
model = Sequential()
model.add(Dense(300, input_shape=(Xtrain.shape[1],)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(Ytrain.shape[1]))
model.add(Activation("softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", 
              metrics=["accuracy"])
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS, verbose=1,
                    validation_data=(Xtest, Ytest))

In [ ]:
# plot loss function
plt.subplot(211)
plt.title("accuracy")
plt.plot(history.history["acc"], color="r", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("loss")
plt.plot(history.history["loss"], color="r", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

In [ ]:
# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

# using the word2vec model
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}

# retrieve the weights from the first dense layer. This will convert
# the input vector from a one-hot sum of two words to a dense 300 
# dimensional representation
W, b = model.layers[0].get_weights()

idx2emb = {}    
for word in word2idx.keys():
    wid = word2idx[word]
    vec_in = ohe.fit_transform(np.array(wid)).todense()
    vec_emb = np.dot(vec_in, W)
    idx2emb[wid] = vec_emb

for word in ["stupid", "alice", "succeeded"]:
    wid = word2idx[word]
    source_emb = idx2emb[wid]
    distances = []
    for i in range(1, vocab_size):
        if i == wid:
            continue
        target_emb = idx2emb[i]
        distances.append(((wid, i), 
                         cosine_distances(source_emb, target_emb)))
    sorted_distances = sorted(distances, key=operator.itemgetter(1))[0:10]
    predictions = [idx2word[x[0][1]] for x in sorted_distances]
    print("{:s} => {:s}".format(word, ", ".join(predictions)))